In [7]:
import os
 

In [8]:
%pwd

'c:\\Users\\DELL\\Kapil\\NLP\\NLP-Text-Summarizer\\research'

In [9]:
os.chdir("../")

In [10]:
%pwd

'c:\\Users\\DELL\\Kapil\\NLP\\NLP-Text-Summarizer'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
%pip install PyYAML


  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl (138 kB)


In [7]:
%pip install ensure

  Using cached ensure-1.0.4-py3-none-any.whl.metadata (10 kB)
Using cached ensure-1.0.4-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pathlib import Path

In [12]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )


        return data_ingestion_config

In [ ]:
%pip install python-box
from box import Box




Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
from pathlib import Path

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-05 06:09:48,365: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-05 06:09:48,374: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 06:09:48,377: INFO: common: created directory at: artifacts]
[2024-02-05 06:09:48,380: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-05 06:09:48,382: INFO: 1434958058: File already exists of size: ~ 7718 KB]
